### Pyspark ML

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('ML').getOrCreate()
spark

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/06/23 12:34:18 WARN Utils: Your hostname, N18719, resolves to a loopback address: 127.0.1.1; using 172.29.16.97 instead (on interface wlp0s20f3)
25/06/23 12:34:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/23 12:34:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
training = spark.read.csv('test1.csv', header=True, inferSchema=True)
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [4]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [5]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [8]:
from pyspark.ml.feature import VectorAssembler

featureassembler = VectorAssembler(inputCols=['age', 'Experience'], outputCol = "Independent Features")
output = featureassembler.transform(training)

In [9]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [11]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [18]:
data = output.select(['Salary', 'Independent Features'])

In [19]:
data.show()

+------+--------------------+
|Salary|Independent Features|
+------+--------------------+
| 30000|         [31.0,10.0]|
| 25000|          [30.0,8.0]|
| 20000|          [29.0,4.0]|
| 20000|          [24.0,3.0]|
| 15000|          [21.0,1.0]|
| 18000|          [23.0,2.0]|
+------+--------------------+



## Liner Regression

In [21]:
from pyspark.ml.regression import LinearRegression

# train test split
train_data, test_data = data.randomSplit([0.75, 0.25])
regressor = LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor = regressor.fit(train_data)

25/06/23 12:50:23 WARN Instrumentation: [811ef482] regParam is zero, which might cause numerical instability and overfitting.


In [22]:
### Coefficients
regressor.coefficients

DenseVector([47.619, 1285.7143])

In [23]:
### Intercepts
regressor.intercept

13619.047619047662

In [24]:
### Prediction
pred_result = regressor.evaluate(test_data)

In [25]:
pred_result.predictions.show()

+------+--------------------+-----------------+
|Salary|Independent Features|       prediction|
+------+--------------------+-----------------+
| 18000|          [23.0,2.0]|17285.71428571428|
| 30000|         [31.0,10.0]|27952.38095238097|
+------+--------------------+-----------------+



In [26]:
pred_result.meanAbsoluteError, pred_result.meanSquaredError

(1380.9523809523762, 2351473.922902466)